In [1]:
TEST_A = "/Users/joregan/Playing/rd_ctm_edit/H5C120171011va"

In [2]:
lines = []
with open(TEST_A) as f:
    for line in f.readlines():
        lines.append(line.strip())

In [3]:
def accept_all(lines):
    outlines = []
    for line in lines:
        parts = line.split(" ")
        if parts[-1] == "cor":
            outlines.append(line)
        elif parts[-1] == "sub":
            parts[4] = parts[6]
            parts[-1] = "cor"
            outlines.append(" ".join(parts))
    return outlines

In [4]:
accept_all(lines)

['2442205210012872721 1 27.86 0.06 Herr 1.0 Herr cor',
 '2442205210012872721 1 28.0 0.48 talman! 1.0 talman! cor',
 '2442205210012872721 1 28.72 0.96 Riksdagsledamöter! 1.0 Riksdagsledamöter! cor',
 '2442205210012872721 1 30.16 0.82 Allianspartierna 1.0 Allianspartierna cor',
 '2442205210012872721 1 31.08 0.56 Moderaterna, 1.0 Moderaterna, cor',
 '2442205210012872721 1 32.08 0.74 Centerpartiet, 1.0 Centerpartiet, cor',
 '2442205210012872721 1 33.02 0.579 Liberalerna 1.0 Liberalerna cor',
 '2442205210012872721 1 33.74 0.079 och 1.0 och cor',
 '2442205210012872721 1 33.88 0.759 Kristdemokraterna 1.0 Kristdemokraterna cor',
 '2442205210012872721 1 34.76 0.399 föreslår 1.0 föreslår cor',
 '2442205210012872721 1 35.22 0.079 som 1.0 som cor',
 '2442205210012872721 1 35.36 0.579 riksdagens 1.0 riksdagens cor',
 '2442205210012872721 1 36.02 0.239 förste 1.0 förste cor',
 '2442205210012872721 1 36.32 0.179 vice 1.0 vice cor',
 '2442205210012872721 1 36.58 0.34 talman 1.0 talman cor',
 '24422052

In [5]:
def ctm_to_timed(lines):
    output = []
    for line in lines:
        parts = line.split(" ")
        start = float(parts[2])
        dur = float(parts[3])
        output.append({
            "start": start,
            "end": start + dur,
            "text": parts[6]
        })
    return output

In [6]:
side_a = ctm_to_timed(accept_all(lines))

In [7]:
phonfile = "/Users/joregan/Playing/rd_phonetic/2442205210012872721_480p.json"

In [8]:
import json
with open(phonfile) as f:
    pieces = json.load(f)

In [9]:
def hf_json_to_timed(data):
    output = []
    for chunk in data["chunks"]:
        output.append({
            "start": chunk["timestamp"][0],
            "end": chunk["timestamp"][1],
            "text": chunk["text"]
        })
    return output

In [10]:
side_b = hf_json_to_timed(pieces)

In [11]:
def prune_to_other(left, right, fudge=0.5):
    safe = left
    working = right
    # if right[0]["start"] > (left[0]["start"] - fudge) and right[-1]["end"] < (left[-1]["end"] + fudge):
    #     safe = right
    #     working = left
    output = []
    for item in working:
        if item["start"] < safe[0]["start"] - fudge:
            continue
        elif item["end"] > safe[-1]["end"] + fudge:
            continue
        else:
            output.append(item)

    return safe, output

In [12]:
new_a, new_b = prune_to_other(side_a, side_b)

In [13]:
new_b[0]

{'start': 27.86, 'end': 27.9, 'text': 'ɑː'}

In [14]:
new_a[0]

{'start': 27.86, 'end': 27.919999999999998, 'text': 'Herr'}

In [15]:
def cost(a, b):
    starts = abs(a["start"] - b["start"])
    ends = abs(a["end"] - b["end"])
    return starts + ends

In [16]:
def in_start_range(a, b, range=0.2):
    return abs(a["start"] - b["start"]) <= range

def in_end_range(a, b, range=0.2):
    return abs(a["end"] - b["end"]) <= range

def in_range(a, b, range=0.2):
    r_start = in_start_range(a, b, range)
    r_end = in_end_range(a, b, range)
    return r_start or r_end


In [17]:
in_range(new_a[0], new_b[-1])

False

In [18]:
in_range(new_a[0], new_b[0])

True

In [19]:
import numpy as np

In [20]:
dist_matrix = np.zeros((len(new_a) + 1, len(new_b) + 1))

In [21]:
for i in range(1, len(new_a) + 1):
    for j in range(1, len(new_b) + 1):
        if not in_range(new_a[i-1], new_b[j-1]):
            continue
        pair_cost = cost(new_a[i-1], new_b[j-1])
        dist_matrix[i, j] = pair_cost
        print(new_a[i-1], new_b[j-1], pair_cost)

{'start': 27.86, 'end': 27.919999999999998, 'text': 'Herr'} {'start': 27.86, 'end': 27.9, 'text': 'ɑː'} 0.019999999999999574
{'start': 27.86, 'end': 27.919999999999998, 'text': 'Herr'} {'start': 28.0, 'end': 28.6, 'text': 'tɑːlman'} 0.8200000000000038
{'start': 28.0, 'end': 28.48, 'text': 'talman!'} {'start': 27.86, 'end': 27.9, 'text': 'ɑː'} 0.7200000000000024
{'start': 28.0, 'end': 28.48, 'text': 'talman!'} {'start': 28.0, 'end': 28.6, 'text': 'tɑːlman'} 0.120000000000001
{'start': 28.72, 'end': 29.68, 'text': 'Riksdagsledamöter!'} {'start': 28.7, 'end': 29.18, 'text': 'rɪksasleːda'} 0.5199999999999996
{'start': 28.72, 'end': 29.68, 'text': 'Riksdagsledamöter!'} {'start': 29.24, 'end': 29.58, 'text': 'møːtœ̞'} 0.620000000000001
{'start': 30.16, 'end': 30.98, 'text': 'Allianspartierna'} {'start': 30.0, 'end': 30.04, 'text': '<pa>'} 1.1000000000000014
{'start': 30.16, 'end': 30.98, 'text': 'Allianspartierna'} {'start': 30.16, 'end': 30.26, 'text': 'al'} 0.7199999999999989
{'start': 30.

In [26]:
s1 = len(new_a) + 1
s2 = len(new_b) + 1

for i in range(s1):
    dist_matrix[i, 0] = float(i)
for j in range(s2):
    dist_matrix[0, j] = float(j)

for i in range(1, s1):
    for j in range(1, s2):
        if not in_range(new_a[i-1], new_b[j-1]):
            continue
        pair_cost = cost(new_a[i-1], new_b[j-1])
        dist_matrix[i, j] = min(
            dist_matrix[i - 1][j] + pair_cost,
            dist_matrix[i][j - 1] + pair_cost,
            dist_matrix[i - 1][j - 1] + pair_cost
        )


In [27]:
import pandas as pd
df = pd.DataFrame(data=dist_matrix,index=[""] + [x["text"] for x in new_a], columns=[""] + [x["text"] for x in new_b])

In [39]:
def falls_between(a1, a2, b):
    if b["end"] <= a2["start"] and b["start"] >= a1["end"]:
        return True
    return False

In [38]:
df

,,ɑː,tɑːlman,rɪksasleːda,møːtœ̞,<pa>,al,aspatiːæɳa,mʊdɑːtœ̞ɔɳa,<pa>,...,fœ̞ːesoː,sɔm,rɪksdɑːɡəns,fœ̞st,viːsə,tɑːlman,<pa>,eva,tareːn,fɪne
,0.0,1.00,2.00,3.00,4.00,5.0,6.00,7.00,8.00,9.00,...,16.000,17.000,18.000,19.000,20.000,21.00,22.0,23.00,24.00,25.00
Herr,1.0,0.02,0.84,0.00,0.00,0.0,0.00,0.00,0.00,0.00,...,0.000,0.000,0.000,0.000,0.000,0.00,0.0,0.00,0.00,0.00
talman!,2.0,0.74,0.14,0.00,0.00,0.0,0.00,0.00,0.00,0.00,...,0.000,0.000,0.000,0.000,0.000,0.00,0.0,0.00,0.00,0.00
Riksdagsledamöter!,3.0,0.00,0.00,0.52,0.62,0.0,0.00,0.00,0.00,0.00,...,0.000,0.000,0.000,0.000,0.000,0.00,0.0,0.00,0.00,0.00
Allianspartierna,4.0,0.00,0.00,0.00,0.00,1.1,0.72,0.16,0.00,0.00,...,0.000,0.000,0.000,0.000,0.000,0.00,0.0,0.00,0.00,0.00
"Moderaterna,",5.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.02,0.88,...,0.000,0.000,0.000,0.000,0.000,0.00,0.0,0.00,0.00,0.00
"Centerpartiet,",6.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,...,0.000,0.000,0.000,0.000,0.000,0.00,0.0,0.00,0.00,0.00
Liberalerna,7.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,...,0.000,0.000,0.000,0.000,0.000,0.00,0.0,0.00,0.00,0.00
och,8.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,...,0.000,0.000,0.000,0.000,0.000,0.00,0.0,0.00,0.00,0.00
Kristdemokraterna,9.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,...,0.000,0.000,0.000,0.000,0.000,0.00,0.0,0.00,0.00,0.00
